In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# --- 1. 드라이버 설정 및 변수 지정 ---
company_name = '(주)카카오페이'
job_data = [] # 모든 페이지의 채용 정보를 누적할 리스트

options = webdriver.ChromeOptions()
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

try:
    # --- 2. 초기 페이지 이동 및 검색, 탭 클릭 과정 ---
    # (이전 단계들과 동일)
    driver.get('https://www.jobkorea.co.kr/')
    print("1. 잡코리아 사이트로 이동했습니다.")

    search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'stext')))
    search_box.clear()
    search_box.send_keys(company_name)
    
    search_button = driver.find_element(By.ID, 'common_search_btn')
    search_button.click()
    print(f"2. '{company_name}'을/를 검색했습니다.")
    time.sleep(3)

    search_keyword = company_name.replace('(주)', '').strip()
    print(f"3. 검색 결과에서 '{search_keyword}' 기업 정보 링크를 찾습니다.")
    company_link_xpath = (
        f"//a[@data-sentry-component='Title' and contains(., '{search_keyword}')]"
        f"/parent::div/preceding-sibling::div//a[contains(., '{search_keyword}')]"
    )
    company_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, company_link_xpath)))
    company_link.click()
    time.sleep(2)

    print("4. 새로 열린 탭으로 전환합니다.")
    driver.switch_to.window(driver.window_handles[-1])
    
    print("5. '기업정보' 버튼을 클릭합니다.")
    company_info_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'girBtn') and .//span[text()='기업정보']]"))
    )
    company_info_button.click()
    time.sleep(2)

    print("6. 새로 열린 '기업정보 상세' 탭으로 전환합니다.")
    driver.switch_to.window(driver.window_handles[-1])

    print("7. '채용' 탭을 클릭합니다.")
    recruit_tab_xpath = "//a[contains(@class, 'company-nav-item') and .//div[@class='name' and text()='채용']]"
    recruit_tab = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, recruit_tab_xpath)))
    recruit_tab.click()
    time.sleep(2)
    
    print("8. '전체' 버튼을 클릭하여 모든 공고를 불러옵니다.")
    all_jobs_button_xpath = "//a[contains(@class, 'btnSort') and starts-with(normalize-space(.), '전체')]"
    all_jobs_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, all_jobs_button_xpath)))
    all_jobs_button.click()
    time.sleep(2)

    
    page_num = 1
    while True:
        print(f"9. 각 페이지마다 공고 크롤링 \n--- {page_num} 페이지 크롤링 시작 ---")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        job_list_container = soup.select_one("div.list.clear")
        if not job_list_container:
            print("   공고 목록 컨테이너를 찾을 수 없어 종료합니다.")
            break
            
        job_items = job_list_container.select("div.list-item")
        if not job_items:
            print("   해당 페이지에 공고가 없습니다.")
            break

        print(f"   {len(job_items)}개의 공고를 수집합니다.")
        for item in job_items:
            status = "마감" if 'end' in item.get('class', []) else "진행중"
            title = item.select_one("dt.tit").get_text(strip=True) if item.select_one("dt.tit") else ""
            conditions = item.select_one("dd.trm").get_text(separator=' / ', strip=True) if item.select_one("dd.trm") else ""
            deadline = item.select_one("dd.func > span.day").get_text(strip=True) if item.select_one("dd.func > span.day") else ""
            job_category = item.select_one("dd.sub").get_text(strip=True) if item.select_one("dd.sub") else ""
            link_tag = item.select_one("a")
            link = "https://www.jobkorea.co.kr" + link_tag['href'] if link_tag else ""

            job_data.append({
                "페이지": page_num, "상태": status, "공고명": title, "지원자격": conditions,
                "마감일": deadline, "직무": job_category, "상세링크": link
            })
        
        # '다음' 버튼 찾기 및 클릭
        try:
            # 더 안정적인 선택자: 클래스 이름으로 '다음' 버튼을 직접 찾음
            next_button_selector = "a.tplBtn.btnPgnNext"
            next_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, next_button_selector))
            )
            # JavaScript로 클릭하는 것이 더 안정적일 수 있습니다.
            driver.execute_script("arguments[0].click();", next_button)
            
            print(f"--- {page_num} 페이지 크롤링 완료. 다음 페이지로 이동합니다. ---")
            page_num += 1
            time.sleep(2) # 페이지 로딩 대기
            
        except (TimeoutException, NoSuchElementException):
            # '다음' 버튼을 더 이상 찾을 수 없으면 마지막 페이지로 간주하고 반복 종료
            print("\n마지막 페이지입니다. 크롤링을 종료합니다.")
            break

except Exception as e:
    print(f"전체 프로세스 중 오류가 발생했습니다: {e}")

finally:
    # --- 최종 단계: 브라우저 종료 및 파일 저장 ---
    driver.quit()
    if job_data:
        df = pd.DataFrame(job_data)
        file_name = f"{company_name}_전체채용공고.csv"
        df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"\n🎉 총 {len(job_data)}개의 공고 정보를 '{file_name}' 파일로 저장했습니다.")
    else:
        print("\n크롤링된 데이터가 없어 파일을 저장하지 않았습니다.")